In [12]:
from pydantic import BaseModel, Field
from typing import List, Union, Dict, Any 
from typing import Annotated, List, Union, Dict, Any
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict

# 테이블의 한 행을 구성하는 값의 타입을 명확히 정의합니다.
TableRow = Dict[str, Union[str, int, float, bool, None]]

class MessagePart(BaseModel):
    """Part의 종류 ('text', 'data-sheet' 등)"""
    type: str = Field(description="Part의 종류. 일반 텍스트는 'text', 표 형태 데이터는 'data-sheet'.")
    text: Union[str, None] = Field(None, description="단순 텍스트 내용")
    # 'Any' 대신 명확하게 정의된 'TableRow' 타입을 사용합니다.
    data: Union[List[TableRow], None] = Field(None, description="표와 같은 구조화된 데이터")

class StructuredResponse(BaseModel):
    """메시지를 구성하는 하나 이상의 파트 배열"""
    parts: List[MessagePart] = Field(description="메시지를 구성하는 하나 이상의 파트 배열")
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph
from pydantic import BaseModel

client = MultiServerMCPClient({
    "sqlserver": {
        "transport": 'streamable_http',
        "url": "http://localhost:5298/mcp"
    }
})
mcp_tools = await client.get_tools()
react_agent = create_react_agent(
    model = "openai:gpt-5-nano",
    tools=mcp_tools,
    prompt="""You are a helpful assistant. Respond in ReAct format:
1. Always provide: Thought: [Your step-by-step reasoning]
2. If a tool is needed, use the tool-calling mechanism to invoke it
3. If no tool is needed but reasoning is incomplete, output: Continue: [next reasoning step or sub-question]
4. If reasoning is complete and no tool is needed, output: Final Answer: [final response]
Every response MUST include a Thought in the content field, even when using tool-calling.
Do NOT use tools unless the query explicitly requires external data.
For reasoning tasks, use Continue for intermediate steps.""",
)


In [14]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
from langchain_core.prompts import ChatPromptTemplate
formatter_prompt = ChatPromptTemplate.from_messages([
    ("system", 
        "You are a data formatting expert. Your task is to analyze the user's final response and convert it into the provided JSON schema. "
        "If the response contains tabular data (like a list of tables), use the 'data-sheet' type. "
        "Otherwise, use the 'text' type."),
    ("human", "Please format the following response:\n\n{final_answer}")
])

# .with_structured_output을 사용하여 LLM이 Pydantic 모델 형식으로 응답하도록 강제
formatter_llm = model.with_structured_output(StructuredResponse)
formatter_chain = formatter_prompt | formatter_llm

In [15]:
async def run_react_agent(state: AgentState):
        """기존 ReAct 에이전트를 실행하는 노드"""
        print("--- 1. ReAct 에이전트 실행 중 ---")
        result = await react_agent.ainvoke(state)
        return {"messages": result["messages"]}

async def format_response(state: AgentState):
        """최종 응답을 StructuredResponse로 변환하는 노드"""
        print("--- 2. 최종 응답 포맷팅 중 ---")
        final_answer_message = state["messages"][-1]
        
        # 'Final Answer:' 접두사 제거
        content = final_answer_message.content
        if "Final Answer:" in content:
            content = content.split("Final Answer:", 1)[1].strip()
            
        # 포맷팅 체인 실행
        structured_result = await formatter_chain.ainvoke({"final_answer": content})
        
        # Pydantic 모델을 딕셔너리로 변환하여 새 메시지로 추가
        # (또는 기존 메시지를 대체할 수도 있습니다)
        formatted_message_content = structured_result.model_dump_json(indent=2)
        
        # 여기서는 마지막 메시지의 content를 덮어씁니다.
        state["messages"][-1].content = formatted_message_content
        print("--- 3. 포맷팅 완료 ---")
        return {"messages": state["messages"]}

In [17]:
workflow = StateGraph(AgentState)
    
# 노드 추가
workflow.add_node("agent", run_react_agent)
workflow.add_node("formatter", format_response)

# 엣지 연결
workflow.set_entry_point("agent")
workflow.add_edge("agent", "formatter")
workflow.set_finish_point("formatter")

# 그래프 컴파일
final_graph = workflow.compile(checkpointer=InMemorySaver())

# --- 6. 최종 그래프 실행 ---

config = {"configurable": {"thread_id": "2"}}
query = {"messages": [{"role": "user", "content": "조회가능한 테이블 목록 알아보고, 해당 테이블 조회해서 결과 알려줘"}]}

async for event in final_graph.astream(query, config=config):
    print(event)
    print("---")

--- 1. ReAct 에이전트 실행 중 ---
{'agent': {'messages': [HumanMessage(content='조회가능한 테이블 목록 알아보고, 해당 테이블 조회해서 결과 알려줘', additional_kwargs={}, response_metadata={}, id='611bfb92-f14d-4d86-a3ec-d70d774fc21b'), AIMessage(content='Thought: 먼저 조회 가능한 테이블 목록을 확인해야 합니다. 그다음 각 테이블에서 상위 5건을 조회해 간단한 결과를 요약해서 알려드리겠습니다. 먼저 테이블 목록 조회 도구를 실행하겠습니다.', additional_kwargs={'tool_calls': [{'id': 'call_7RvjSXHDI7Gg1O5ID6OCebJc', 'function': {'arguments': '{}', 'name': 'get_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 847, 'prompt_tokens': 339, 'total_tokens': 1186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CHNBKDAozRqKOV698ZhSB4eM4IKME', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': No